In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

#Ploting
import matplotlib.pyplot as plt
import seaborn as sns

#Preprocessing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

#metrics
from sklearn.metrics import classification_report

#Transfer Learning
from tensorflow.keras.applications import VGG16

In [4]:
file_names = []
classes = []
root_path = '../input/fruit-and-vegetable-image-recognition/train'
for folder in os.listdir(root_path):
    new_path = os.path.join(root_path,folder)
    for file in os.listdir(new_path):
        file_names.append(os.path.join(new_path,file))
        classes.append(folder)

data = pd.DataFrame({'filenames':file_names,'class':classes})

In [5]:
data['class'].value_counts()

In [6]:
fig = plt.figure(figsize=(15,7))
sns.countplot(x='class',data=data)

In [7]:
data_gen = ImageDataGenerator(rescale=1./255,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             rotation_range = 20,
                             horizontal_flip = True,
                             vertical_flip = True,
                             fill_mode = 'nearest')
test_data_gen = ImageDataGenerator(rescale=1./255)

In [11]:
train_generator = data_gen.flow_from_directory('../input/fruit-and-vegetable-image-recognition/train',
                                              target_size = (150,150),
                                              color_mode = 'rgb',
                                              class_mode = 'categorical',
                                              shuffle = True)
valid_generator = data_gen.flow_from_directory('../input/fruit-and-vegetable-image-recognition/validation',
                                              target_size = (150,150),
                                              color_mode = 'rgb',
                                              class_mode = 'categorical',
                                              shuffle = True)
test_generator = test_data_gen.flow_from_directory('../input/fruit-and-vegetable-image-recognition/test',
                                              target_size = (150,150),
                                              color_mode = 'rgb',
                                              class_mode = 'categorical',
                                              shuffle = True)

In [12]:
fig = plt.figure(figsize=(20,10))
ind = np.random.randint(3000)
img = data.iloc[ind,0]

img = image.load_img(img)

img_array = image.img_to_array(img)

img_array = np.expand_dims(img_array,axis=0)

img_a = data_gen.flow(img_array)
plt.subplot(1,4,1)
plt.imshow(img)
plt.title(f'Original | {data.iloc[ind,1]}')
i = 2
for im in img_a:
    plt.subplot(1,4,i)
    plt.imshow(image.array_to_img(im[0]))
    plt.title("Augmented")
    i += 1
    if i % 5 == 0:
        break  

In [13]:
def show_sample_images(gne): 
    t_dict = gne.class_indices
    classes = list(t_dict.keys())
    images,labels = next(gne)
    plt.figure(figsize=(20,20))
    for i in range(25):
        plt.subplot(5,5,i+1)
        img = images[i]
        plt.imshow(img)
        index = np.argmax(labels[i])
        class_name = classes[index]
        plt.title(class_name)
        plt.axis('off')
    plt.show()

In [14]:
show_sample_images(train_generator)

In [16]:
early_stop = EarlyStopping(monitor='val_loss',patience=3,min_delta = 0.0001)

In [19]:
model = Sequential()

model.add(Conv2D(128,(3,3),padding='same',activation='relu',input_shape=(150,150,3)))
model.add(MaxPool2D(2,2))

model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(2,2))

model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(2,2))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dense(36,activation='softmax'))


model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,validation_data=valid_generator,epochs=100,callbacks=[early_stop])

In [ ]:
model.evaluate(test_generator)

In [ ]:
predictions = model.predict(test_generator)
pred_label = np.argmax(predictions,axis=1)

In [ ]:
t_dict = test_generator.class_indices
classes = list(t_dict.keys())
plt.figure(figsize=(25,20))

images,labels = next(test_generator)

for i in range(30):
    plt.subplot(6,5,i+1)
    img = images[i]
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array,axis=0)
    prediction = np.argmax(model.predict(img_array))
    plt.imshow(img)
    
    index = np.argmax(labels[i])
    class_name = classes[index]
    prediction = model.predict(img_array)
    plt.title(f'ground truth : {class_name} | prediction {classes[np.argmax(prediction[0],axis=0)]}')
    plt.axis('off')
plt.show()